## Imports

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore') #, module = "matplotlib\..*")
%matplotlib inline

## Exploratory data exploration

In [2]:
data_path = "../dataset/regularite-mensuelle-tgv-aqst.csv" 
df_base = pd.read_csv(data_path, sep=",") #, index_col=0)
# df_base.to_csv("regularite-mensuelle-tgv-aqst.csv", index=False)
df_base.head()

,date,service,gare_depart,gare_arrivee,duree_moyenne,nb_train_prevu,nb_annulation,commentaire_annulation,nb_train_depart_retard,retard_moyen_depart,...,nb_train_retard_sup_15,retard_moyen_trains_retard_sup15,nb_train_retard_sup_30,nb_train_retard_sup_60,prct_cause_externe,prct_cause_infra,prct_cause_gestion_trafic,prct_cause_materiel_roulant,prct_cause_gestion_gare,prct_cause_prise_en_charge_voyageurs
0,2018-01,National,BORDEAUX ST JEAN,PARIS MONTPARNASSE,141,870,5,NaN,289,11.247809,...,110,6.511118,44,8,36.134454,31.092437,10.924370,15.966387,5.042017,0.840336
1,2018-01,National,LA ROCHELLE VILLE,PARIS MONTPARNASSE,165,222,0,NaN,8,2.875000,...,22,5.696096,5,0,15.384615,30.769231,38.461538,11.538462,3.846154,0.000000
2,2018-01,National,PARIS MONTPARNASSE,QUIMPER,220,248,1,NaN,37,9.501351,...,26,7.548387,17,7,26.923077,38.461538,15.384615,19.230769,0.000000,0.000000
3,2018-01,National,PARIS MONTPARNASSE,ST MALO,156,102,0,NaN,12,19.912500,...,8,6.724757,6,4,23.076923,46.153846,7.692308,15.384615,7.692308,0.000000
4,2018-01,National,PARIS MONTPARNASSE,ST PIERRE DES CORPS,61,391,2,NaN,61,7.796995,...,17,3.346487,6,0,21.212121,42.424242,9.090909,21.212121,6.060606,0.000000


In [3]:
df_base.sort_values(by=["retard_moyen_tous_trains_arrivee"], ascending=True)[["gare_depart", "gare_arrivee", "nb_train_depart_retard", "retard_moyen_depart", "retard_moyen_arrivee", "retard_moyen_tous_trains_arrivee"]]

,gare_depart,gare_arrivee,nb_train_depart_retard,retard_moyen_depart,retard_moyen_arrivee,retard_moyen_tous_trains_arrivee
2989,STRASBOURG,NANTES,6,1.244444,0.000000,-472.638889
3007,LYON PART DIEU,MONTPELLIER,39,14.598718,35.881111,-173.076970
3213,PARIS EST,FRANCFORT,111,7.517718,31.472222,-163.041983
2889,NIMES,PARIS LYON,190,8.765614,-40.109259,-151.291008
2886,MONTPELLIER,PARIS LYON,189,4.910406,-30.512500,-150.562114
...,...,...,...,...,...,...
787,LYON PART DIEU,RENNES,79,15.840717,58.558681,36.816870
3525,PARIS LYON,ZURICH,4,0.216667,26.469444,37.838235
3522,STUTTGART,PARIS EST,1,4.416667,61.816667,61.816667
3508,STRASBOURG,PARIS EST,3,10.522222,38.816667,83.388889


In [4]:
df_base.loc[df_base.retard_moyen_tous_trains_arrivee > df_base.retard_moyen_arrivee].sort_values(by=["retard_moyen_tous_trains_arrivee"])[["gare_depart", "gare_arrivee", "nb_train_depart_retard", "retard_moyen_depart", "retard_moyen_arrivee", "retard_moyen_tous_trains_arrivee"]]

,gare_depart,gare_arrivee,nb_train_depart_retard,retard_moyen_depart,retard_moyen_arrivee,retard_moyen_tous_trains_arrivee
6407,PARIS LYON,LAUSANNE,3,46.188889,0.000000,0.059420
3589,ANNECY,PARIS LYON,0,0.000000,0.000000,0.106667
6868,TOURCOING,BORDEAUX ST JEAN,0,0.000000,0.000000,0.186022
2912,MARSEILLE ST CHARLES,MADRID,5,0.146667,0.000000,0.205556
5026,NANTES,STRASBOURG,3,2.377778,0.000000,0.216667
...,...,...,...,...,...,...
6692,MADRID,MARSEILLE ST CHARLES,0,0.000000,0.000000,21.289506
3428,ZURICH,PARIS LYON,1,16.116667,11.250000,22.825000
6838,MADRID,MARSEILLE ST CHARLES,0,0.000000,0.000000,28.008602
3525,PARIS LYON,ZURICH,4,0.216667,26.469444,37.838235


In [5]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8154 entries, 0 to 8153
Data columns (total 26 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   date                                  8154 non-null   object 
 1   service                               8154 non-null   object 
 2   gare_depart                           8154 non-null   object 
 3   gare_arrivee                          8154 non-null   object 
 4   duree_moyenne                         8154 non-null   int64  
 5   nb_train_prevu                        8154 non-null   int64  
 6   nb_annulation                         8154 non-null   int64  
 7   commentaire_annulation                0 non-null      float64
 8   nb_train_depart_retard                8154 non-null   int64  
 9   retard_moyen_depart                   8154 non-null   float64
 10  retard_moyen_tous_trains_depart       8154 non-null   float64
 11  commentaire_retar

In [6]:
df_base.describe()

,duree_moyenne,nb_train_prevu,nb_annulation,commentaire_annulation,nb_train_depart_retard,retard_moyen_depart,retard_moyen_tous_trains_depart,commentaire_retards_depart,nb_train_retard_arrivee,retard_moyen_arrivee,...,nb_train_retard_sup_15,retard_moyen_trains_retard_sup15,nb_train_retard_sup_30,nb_train_retard_sup_60,prct_cause_externe,prct_cause_infra,prct_cause_gestion_trafic,prct_cause_materiel_roulant,prct_cause_gestion_gare,prct_cause_prise_en_charge_voyageurs
count,8154.000000,8154.000000,8154.000000,0.0,8154.000000,8154.000000,8154.000000,0.0,8154.000000,8154.000000,...,8154.000000,8154.000000,8154.000000,8154.000000,8154.000000,8154.000000,8154.000000,8154.000000,8154.000000,8154.000000
mean,169.860314,257.198308,11.349644,NaN,88.434265,11.205745,3.076737,NaN,34.516066,33.950971,...,24.434511,32.193823,11.642016,4.245892,23.266846,21.888418,19.052124,18.995179,6.714513,7.127315
std,87.390774,173.815703,26.544553,NaN,90.384803,12.476780,5.432676,NaN,29.729349,15.702663,...,21.400761,20.195926,11.225104,4.901032,16.805192,15.692521,14.947548,14.256533,8.080759,10.249453
min,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,-229.269444,NaN,0.000000,-40.109259,...,0.000000,-4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,100.000000,142.000000,0.000000,NaN,22.000000,5.128410,1.140258,NaN,13.000000,24.636941,...,9.000000,12.140350,4.000000,1.000000,11.764706,11.538462,8.695652,9.523810,0.000000,0.000000
50%,164.000000,221.000000,3.000000,NaN,55.000000,9.036447,2.248271,NaN,27.000000,32.441236,...,19.000000,34.932765,9.000000,3.000000,20.967742,20.000000,16.949153,17.165179,5.000000,3.703704
75%,221.000000,333.000000,10.000000,NaN,129.000000,14.219375,3.903651,NaN,47.000000,41.346717,...,33.000000,44.268749,16.000000,6.000000,32.307692,30.000000,26.923077,26.086957,10.000000,10.000000
max,786.000000,1075.000000,297.000000,NaN,596.000000,316.188095,84.516667,NaN,376.000000,299.600000,...,312.000000,299.600000,202.000000,71.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


### Objectif 1 : Prediction du retard des trains

In [7]:
columns = df_base.columns.to_list()
num_cols = ["annee", "mois", "duree_moyenne", "nb_train_prevu", "retard_moyen_tous_trains_arrivee"]
cat_cols = ["service", "gare_depart", "gare_arrivee"]

df_1 = df_base[columns[:6] + ["retard_moyen_tous_trains_arrivee"]].copy(deep=True)
df_1[['annee', 'mois']] = df_1['date'].str.split('-', n=1, expand=True)
df_1.drop(['date'], axis=1, inplace=True)
df_1.drop_duplicates(inplace=True)
df_1.annee = pd.to_numeric(df_1.annee)
df_1.mois = pd.to_numeric(df_1.mois)

df_1.describe(include="all")

,service,gare_depart,gare_arrivee,duree_moyenne,nb_train_prevu,retard_moyen_tous_trains_arrivee,annee,mois
count,8154,8154,8154,8154.000000,8154.000000,8154.000000,8154.000000,8154.000000
unique,2,59,59,NaN,NaN,NaN,NaN,NaN
top,National,PARIS LYON,PARIS LYON,NaN,NaN,NaN,NaN,NaN
freq,7142,1649,1649,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,169.860314,257.198308,5.668612,2020.240986,6.236326
std,NaN,NaN,NaN,87.390774,173.815703,7.971691,1.608478,3.441466
min,NaN,NaN,NaN,0.000000,0.000000,-472.638889,2018.000000,1.000000
25%,NaN,NaN,NaN,100.000000,142.000000,3.129384,2019.000000,3.000000
50%,NaN,NaN,NaN,164.000000,221.000000,4.962422,2020.000000,6.000000
75%,NaN,NaN,NaN,221.000000,333.000000,7.673542,2022.000000,9.000000


#### Basic preprocessing

In [8]:
# TRAIN-TEST SPLIT
df_1_train = df_1.loc[df_1.annee < 2023]
df_1_test = df_1.loc[df_1.annee == 2023]

In [9]:
# OUTLIERS

# valeur min de -472.638889 pour retard_moyen_tous_trains_arrivee...
# sns.histplot(df_1_train['retard_moyen_tous_trains_arrivee'])
# df_1_train.sort_values(by=["retard_moyen_tous_trains_arrivee"], ascending=True)

# Méthode de Tuckey pour les outliers

outlier_cols = ["duree_moyenne", "nb_train_prevu", "retard_moyen_tous_trains_arrivee"]

def smooth_outliers(val, outlier_step):
    if val < Q1-outlier_step:
        val = Q1-outlier_step
    elif val > Q3+outlier_step:
        val = Q3+outlier_step
    return val


for df in [df_1_train, df_1_test]:
    for col_name in outlier_cols:
        # 1st quartile (25%)
        Q1 = np.percentile(df[col_name], 25)
        # 3rd quartile (75%)
        Q3 = np.percentile(df[col_name],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        # outlier step
        outlier_step = 1.5 * IQR
        df[col_name] = df[col_name].apply(lambda x: smooth_outliers(x, outlier_step))

df_1_train.describe(include="all")

,service,gare_depart,gare_arrivee,duree_moyenne,nb_train_prevu,retard_moyen_tous_trains_arrivee,annee,mois
count,7428,7428,7428,7428.000000,7428.000000,7428.000000,7428.000000,7428.000000
unique,2,59,59,NaN,NaN,NaN,NaN,NaN
top,National,PARIS LYON,PARIS LYON,NaN,NaN,NaN,NaN,NaN
freq,6500,1499,1499,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,168.766290,247.965199,5.584730,2019.971325,6.503770
std,NaN,NaN,NaN,85.040127,157.479909,3.559100,1.422420,3.451503
min,NaN,NaN,NaN,0.000000,0.000000,-3.635839,2018.000000,1.000000
25%,NaN,NaN,NaN,99.000000,138.000000,3.064211,2019.000000,4.000000
50%,NaN,NaN,NaN,163.000000,218.000000,4.870460,2020.000000,7.000000
75%,NaN,NaN,NaN,221.000000,327.000000,7.530911,2021.000000,10.000000


In [10]:
# VALEURS MANQUANTES
print(f"Train df : {df_1_train.isna().sum().sum() == 0}") # no missing values
print(f"Test df : {df_1_test.isna().sum().sum() == 0}") # no missing values

Train df : True
Test df : True


In [11]:
# CREATE = FEATURE ENGINEERING

# Simple binary ohe for the "service" feature
for df in [df_1_train, df_1_test]:
    df.service = df.service.apply(lambda x: 0 if x == "National" else 1)

In [12]:
# For the "gare_depart" and "gare_arrivee" features:
# Applying simple OHE will give way to many sparse features (more than 100...)
# https://annahava.medium.com/too-many-categories-how-to-deal-with-categorical-features-of-high-cardinality-d4563cfe62d6

# We need to find a way to group these features : clusters !
# - groupe train stations by magnitude of the associated city ?
# - by vague geographic location ?
# - merge both features into one and then applying some clustering ?
# - ...

In [ ]:
# Add geo features
# https://medium.com/@amorrison_58444/geocoding-with-the-openstreetmap-api-and-geopy-325633980a15
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="adrico")

def get_coordinates(station_names):
    station_coords = {}
    for station_name in station_names:
        # city_name = station_name.split(" ")[0]
        if station_name in station_coords.keys():
            break
        location_variable = geolocator.geocode(f"{station_name}, France", limit=1, exactly_one=False)
        # print(location_variable)
        for info in location_variable:
            # print(info.raw)
            # if info.raw["addresstype"] == "city": # maybe not necessary...
            lat = info.raw["lat"]
            lon = info.raw["lon"]
            station_coords[station_name] = (lat, lon)
            # print(station_name, lat, lon)
                # break
    return station_coords

station_names = sorted(list(set(list(df_1_train.gare_depart.unique()) + list(df_1_train.gare_arrivee.unique()))))
station_coords = get_coordinates(station_names)

for station_type in ["depart", "arrivee"]:
    df_1_train[f"geo_infos_{station_type}"] = df_1_train[f"gare_{station_type}"].apply(lambda x:station_coords[x])


In [19]:
# https://medium.com/datasciencearth/map-visualization-with-folium-d1403771717

import folium

colors = ["green", "yellow", "orange", "red", "black"]
max_delay = max(df_1_train.retard_moyen_tous_trains_arrivee)
for station_type in ["depart", "arrivee"]:
    sncf_map = folium.Map(location=[47,2], zoom_start=5, tiles="cartodb positron")
    df_1_train_map = pd.DataFrame(df_1_train.groupby([f"gare_{station_type}", "geo_infos_depart", "geo_infos_arrivee"])["retard_moyen_tous_trains_arrivee"].mean()).reset_index(level=[0,1,2]).copy(deep=True)

    for idx, row in df_1_train_map.iterrows():
        line_coords = []
        geo_col_name = f"geo_infos_{station_type}"
        station_col_name = f"gare_{station_type}"
        lat, lon = row[geo_col_name]
        line_coords.append((lat, lon))
        station = row[station_col_name]
        color = colors[max(0,int(len(colors)*row["retard_moyen_tous_trains_arrivee"]/max_delay)-1)]
        folium.Marker(location=[lat,lon], popup = station, icon=folium.Icon(color=color)).add_to(sncf_map)
        # FOR SOME REASON, POLYLINE CRASHES THE KERNEL...
        # folium.PolyLine(line_coords,
        #                 color=colors[int(len(colors)*row["retard_moyen_tous_trains_arrivee"]/max_delay)],
        #                 weight=15,
        #                 opacity=0.8).add_to(train_map)
    sncf_map.save(f"train_map_{station_type}.html")

In [ ]:
# CONVERT
dummy_cols = ["service", "gare_depart", "gare_arrivee"]

for df in [df_1_train, df_1_test]:
    df = pd.get_dummies(df, prefix="ohe", columns=dummy_cols, drop_first=True)
df

,duree_moyenne,nb_train_prevu,retard_moyen_tous_trains_arrivee,annee,mois,ohe_1,ohe_ANGERS SAINT LAUD,ohe_ANGOULEME,ohe_ANNECY,ohe_ARRAS,...,ohe_ST PIERRE DES CORPS,ohe_STRASBOURG,ohe_STUTTGART,ohe_TOULON,ohe_TOULOUSE MATABIAU,ohe_TOURCOING,ohe_TOURS,ohe_VALENCE ALIXAN TGV,ohe_VANNES,ohe_ZURICH
7428,189.0,150.000,13.333464,2023,1,True,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
7429,247.0,129.000,7.943927,2023,1,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
7430,140.0,757.625,6.065343,2023,1,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7431,180.0,334.000,9.004088,2023,1,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7432,99.0,252.000,9.226940,2023,1,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8149,114.0,492.000,7.689898,2023,6,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8150,273.0,215.000,14.824264,2023,6,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8151,78.0,192.000,8.650349,2023,6,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8152,133.0,440.000,12.765753,2023,6,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
